# Heat Transmission Calculation Notebook

This notebook sets up a Python environment and demonstrates interactive forms and dropdowns for heat transmission calculations using Jupyter widgets.

## Install and Import Jupyter Widgets

The following cell ensures `ipywidgets` is installed and imports the necessary modules for interactive widgets.

In [1]:
# Install ipywidgets if not already installed (uncomment if needed)
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

## Warmtedoorgangscoëfficiënt (U-waarde)

$$U = \frac{1}{R_e + R_c + R_i} \quad \left[\frac{W}{m^2 \cdot K}\right]$$

| Symbool | Omschrijving | Standaardwaarde |
|---------|-------------|-----------------|
| $R_i$ | Warmte-overgangsweerstand binnenzijde | 0,13 m²·K/W |
| $R_e$ | Warmte-overgangsweerstand buitenzijde | 0,04 m²·K/W |
| $R_c$ | Warmteweerstand constructie | $\sum d / \lambda$ |

Voeg lagen toe via de knop hieronder. Per laag kies je een materiaal uit de lijst of vul je de R-waarde handmatig in (voor bijv. luchtspouw).


In [2]:
import json
import importlib
import ipywidgets as widgets
from IPython.display import display, HTML

import heat_calc
importlib.reload(heat_calc)                          # pick up any edits without restarting kernel
from heat_calc import SURFACE_R, LayerWidget

# ── Load material data ────────────────────────────────────────────────────────
with open('material_properties.json', 'r', encoding='utf-8') as f:
    materials = json.load(f)

# ── UI state ──────────────────────────────────────────────────────────────────
layers     = []
layers_box = widgets.VBox([])
out        = widgets.Output()

ri_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Binnenzijde  —  Ri = 0,13 m²·K/W',
    description='Ri (binnen):',
    layout=widgets.Layout(width='340px')
)
re_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Buitenzijde  —  Re = 0,04 m²·K/W',
    description='Re (buiten):',
    layout=widgets.Layout(width='340px')
)
ri_dd.observe(lambda _: refresh(), names='value')
re_dd.observe(lambda _: refresh(), names='value')

add_btn = widgets.Button(
    description='＋ Voeg laag toe', button_style='primary',
    layout=widgets.Layout(width='160px')
)


def refresh(*_):
    """Recompute totals and render the result table."""
    ri = SURFACE_R[ri_dd.value]
    re = SURFACE_R[re_dd.value]

    with out:
        out.clear_output(wait=True)

        rows_html = ''
        total_d   = 0.0
        total_rc  = 0.0

        rows_html += (
            f'<tr class="surface"><td>lucht (binnen)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{ri:.2f}</td></tr>\n'
        )

        for layer in layers:
            info  = layer.row_info()
            r     = info['R']
            d     = info['d']
            d_str = f'{d:.3f}' if isinstance(d, float) else '—'
            r_str = f'{r:.3f}' if r is not None else '?'
            if isinstance(d, float): total_d  += d
            if r is not None:        total_rc += r
            rows_html += (
                f'<tr><td>{info["naam"]}</td>'
                f'<td>{d_str}</td><td>{info["lam"]}</td>'
                f'<td>{r_str}</td><td>—</td></tr>\n'
            )

        rows_html += (
            f'<tr class="surface"><td>lucht (buiten)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{re:.2f}</td></tr>\n'
        )

        total_r = ri + total_rc + re
        u       = 1.0 / total_r if total_r > 0 else None
        u_str   = f'{u:.3f}' if u is not None else '?'
        d_tot   = f'{total_d:.3f}' if total_d > 0 else '—'

        html = f"""
<style>
  .utbl {{ border-collapse:collapse; font-family:sans-serif;
           font-size:13px; min-width:680px; }}
  .utbl th {{ background:#2c5f8a; color:white;
              padding:6px 12px; text-align:left; }}
  .utbl td {{ padding:5px 12px; border-bottom:1px solid #ddd; }}
  .utbl tr:not(.total-row):not(.u-row):hover td {{ background:#f0f7ff; }}
  .utbl .surface td  {{ color:#555; font-style:italic; }}
  .utbl .total-row td {{ font-weight:bold; background:#e8f0fe;
                         border-top:2px solid #2c5f8a; }}
  .utbl .u-row td   {{ font-weight:bold; background:#2c5f8a;
                       color:white; font-size:15px; }}
</style>
<table class="utbl">
  <tr>
    <th>Materiaal / Laag</th>
    <th>d [m]</th>
    <th>λ [W/(m·K)]</th>
    <th>R = d/λ [m²·K/W]</th>
    <th>Ri &amp; Re [m²·K/W]</th>
  </tr>
  {rows_html}
  <tr class="total-row">
    <td>TOTAAL</td>
    <td>{d_tot}</td>
    <td>—</td>
    <td>{total_rc:.3f}</td>
    <td>{ri + re:.2f}</td>
  </tr>
  <tr class="u-row">
    <td colspan="5">
      U = 1 / (R<sub>i</sub> {ri:.2f} + R<sub>c</sub> {total_rc:.3f}
              + R<sub>e</sub> {re:.2f})
      &nbsp;=&nbsp; <b>{u_str} W/(m²·K)</b>
    </td>
  </tr>
</table>"""
        display(HTML(html))


def add_layer(_=None):
    layer = LayerWidget(materials, update_cb=refresh, remove_cb=remove_layer)
    layers.append(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


def remove_layer(layer):
    layers.remove(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


add_btn.on_click(add_layer)

display(widgets.VBox([
    widgets.HTML('<b>Overgangsweerstanden</b>'),
    widgets.HBox([ri_dd, re_dd]),
    widgets.HTML('<b style="margin-top:10px;display:block">Constructielagen (Rc)</b>'),
    layers_box,
    add_btn,
    widgets.HTML('<b style="margin-top:10px;display:block">Resultaat</b>'),
    out,
]))

# Start with one empty layer
add_layer()


## Correctiefactoren f_k, f_ia,k, f_ig,k

Bereken de correctiefactoren voor warmtetransmissieverlies op basis van de aangrenzende situatie:

| # | Situatie | Factor |
|---|---------|--------|
| 1 | Buitenlucht | f_k |
| 2 | Aangrenzend gebouw | f_ia,k |
| 3 | Verwarmde ruimte (zelfde woning) | f_ia,k |
| 4 | Onverwarmde ruimte | f_k |
| 5 | Grond | f_ig,k · f_gw |

Selecteer hieronder de situatie en vul de benodigde invoervelden in. De correctiefactor wordt automatisch berekend.


In [6]:
import importlib
import ipywidgets as widgets
from IPython.display import display, HTML

import fk_calc
importlib.reload(fk_calc)

# ── Lookup data for dropdowns ─────────────────────────────────────────────────
heating_systems = fk_calc.list_heating_systems()
hs_options = {s['omschrijving']: s['id'] for s in heating_systems}
hs_list = list(hs_options.keys())

room_types_woon = fk_calc.list_room_types('woonfunctie')
room_types_senior = fk_calc.list_room_types('seniorenwoningen_verzorgingstehuizen')

# ── Layout constants ──────────────────────────────────────────────────────────
_LBL    = {'description_width': '220px'}   # uniform label width → inputs align
_W_SML  = widgets.Layout(width='320px')    # numeric fields placed side-by-side
_W_MED  = widgets.Layout(width='560px')    # most dropdowns / num fields
_W_WIDE = widgets.Layout(width='700px')    # dropdowns with long option text

# ── Shared widgets ────────────────────────────────────────────────────────────
scenario_dd = widgets.Dropdown(
    options=['Buitenlucht', 'Aangrenzend gebouw',
             'Verwarmde ruimte (zelfde woning)',
             'Onverwarmde ruimte \u2013 bekende temperatuur',
             'Onverwarmde ruimte \u2013 onbekende temperatuur',
             'Grond'],
    description='Situatie:',
    layout=widgets.Layout(width='560px'),
    style={'description_width': '80px'},
)

theta_i_input = widgets.BoundedFloatText(value=22, min=-50, max=50, step=0.5,
    description='Binnentemperatuur [°C]:', layout=_W_SML, style=_LBL)
theta_e_input = widgets.BoundedFloatText(value=-10, min=-50, max=50, step=0.5,
    description='Buitentemperatuur [°C]:', layout=_W_SML, style=_LBL)

# ── Scenario 1: Buitenlucht ──────────────────────────────────────────────────
buitenlucht_bouwdeel = widgets.Dropdown(
    options=['Buitenwand', 'Schuin dak', 'Vloer boven buitenlucht', 'Plat dak'],
    description='Bouwdeel:', layout=_W_MED, style=_LBL)
buitenlucht_hs = widgets.Dropdown(options=hs_list,
    description='Verwarmingssysteem:', layout=_W_WIDE, style=_LBL)
buitenlucht_heated = widgets.Checkbox(value=False,
    description='Verwarmd vlak (wand-/vloerverwarming)',
    layout=widgets.Layout(width='340px'))

# ── Scenario 2: Aangrenzend gebouw ──────────────────────────────────────────
ag_bouwdeel = widgets.Dropdown(options=['Wand', 'Vloer', 'Plafond'],
    description='Bouwdeel:', layout=_W_MED, style=_LBL)
ag_theta_b = widgets.BoundedFloatText(value=20, min=-50, max=50, step=0.5,
    description='Temperatuur aangrenzend [°C]:', layout=_W_MED, style=_LBL)
ag_hs = widgets.Dropdown(options=hs_list,
    description='Verwarmingssysteem:', layout=_W_WIDE, style=_LBL)
ag_heated = widgets.Checkbox(value=False, description='Verwarmd vlak',
    layout=widgets.Layout(width='200px'))

# ── Scenario 3: Verwarmde ruimte ─────────────────────────────────────────────
vr_bouwdeel = widgets.Dropdown(options=['Wand', 'Vloer', 'Plafond'],
    description='Bouwdeel:', layout=_W_MED, style=_LBL)
vr_theta_a = widgets.Dropdown(
    options=[(f'{r["omschrijving"]} ({r["theta_i"]} \u00b0C)', r['theta_i'])
             for r in room_types_woon],
    description='Aangrenzende ruimte:', layout=_W_WIDE, style=_LBL)
vr_override = widgets.Checkbox(value=False,
    description='Temperatuur handmatig invoeren',
    layout=widgets.Layout(width='280px'))
vr_theta_manual = widgets.BoundedFloatText(value=20, min=-50, max=50, step=0.5,
    description='Handmatige temperatuur [°C]:', layout=_W_MED, style=_LBL)
vr_hs_own = widgets.Dropdown(options=hs_list,
    description='Verw. eigen ruimte:', layout=_W_WIDE, style=_LBL)
vr_hs_adj = widgets.Dropdown(options=hs_list,
    description='Verw. aangrenzende ruimte:', layout=_W_WIDE, style=_LBL)
vr_heated = widgets.Checkbox(value=False, description='Verwarmd vlak',
    layout=widgets.Layout(width='200px'))

# ── Scenario 4a: Onverwarmd – bekende temperatuur ────────────────────────────
ob_bouwdeel = widgets.Dropdown(options=['Wand', 'Vloer', 'Plafond'],
    description='Bouwdeel:', layout=_W_MED, style=_LBL)
ob_theta_a = widgets.BoundedFloatText(value=5, min=-50, max=50, step=0.5,
    description='Temperatuur onverwarmd [°C]:', layout=_W_MED, style=_LBL)
ob_hs = widgets.Dropdown(options=hs_list,
    description='Verwarmingssysteem:', layout=_W_WIDE, style=_LBL)
ob_heated = widgets.Checkbox(value=False, description='Verwarmd vlak',
    layout=widgets.Layout(width='200px'))

# ── Scenario 4b: Onverwarmd – onbekende temperatuur ──────────────────────────
oo_doel = widgets.ToggleButtons(options=['Warmteverlies', 'Tijdconstante'],
    description='Doel:', style={'description_width': '40px'})
oo_ruimte = widgets.Dropdown(
    options=['Vertrek', 'Ruimte onder dak', 'Verkeersruimte', 'Kruipruimte'],
    description='Type ruimte:', layout=_W_MED, style=_LBL)
# Value is tuple (aantal_gevels: int, buitendeur: bool | None) — buitendeur embedded
oo_gevels = widgets.Dropdown(
    options=[
        ('1 externe scheidingsconstructie / buitenwand',                   (1, None)),
        ('2 externe scheidingsconstructies \u2013 zonder buitendeur',       (2, False)),
        ('2 externe scheidingsconstructies \u2013 met buitendeur',          (2, True)),
        ('3 of meer externe scheidingsconstructies',                        (3, None)),
    ],
    description='Wat voor gevel:',
    layout=_W_WIDE, style=_LBL)
oo_daktype = widgets.Dropdown(
    options={
        'Pannendak zonder folie (hoog infiltratievoud)': 'pannendak_zonder_folie',
        'Overige niet-ge\u00efsoleerde daken':            'niet_geisoleerd',
        'Ge\u00efsoleerde daken':                         'geisoleerd',
    },
    description='Daktype:', layout=_W_WIDE, style=_LBL)
oo_buitenwanden = widgets.Checkbox(value=True, description='Buitenwanden aanwezig',
    layout=widgets.Layout(width='240px'))
oo_ventilatievoud = widgets.BoundedFloatText(value=0.3, min=0, max=50, step=0.1,
    description='Ventilatievoud:', layout=_W_MED, style=_LBL)
oo_a_opening = widgets.BoundedFloatText(value=0.003, min=0, max=1, step=0.001,
    description='A_opening / V:', layout=_W_MED, style=_LBL)
oo_opening_mm2 = widgets.BoundedFloatText(value=800, min=0, max=10000, step=50,
    description='Opening [mm\u00b2/m\u00b2]:', layout=_W_MED, style=_LBL)
oo_tijdconst_ruimte = widgets.Dropdown(
    options={
        'Kelder':                             'kelder',
        'Stallingsruimte':                    'stallingsruimte',
        'Kruipruimte / serre / trappenhuis':  'kruipruimte_serre_trappenhuis',
    },
    description='Type ruimte:', layout=_W_MED, style=_LBL)

# ── Scenario 5: Grond ────────────────────────────────────────────────────────
gr_bouwdeel = widgets.Dropdown(options=['Wand', 'Vloer'],
    description='Bouwdeel:', layout=_W_MED, style=_LBL)
gr_theta_me = widgets.BoundedFloatText(value=10.5, min=-20, max=30, step=0.1,
    description='Jaarl. gem. buitentemp. [°C]:', layout=_W_MED, style=_LBL)
gr_hs = widgets.Dropdown(options=hs_list,
    description='Verwarmingssysteem:', layout=_W_WIDE, style=_LBL)
gr_heated = widgets.Checkbox(value=False, description='Verwarmd vlak op grond',
    layout=widgets.Layout(width='260px'))
# Value is the f_gw factor directly: 1.00 (no groundwater effect) or 1.15 (groundwater present)
gr_gwstand = widgets.Dropdown(
    options=[
        ('Geen grondwater \u2013 grondwater \u2265 1 m onder vloer  \u2192  f_gw = 1,00', 1.00),
        ('Grondwater aanwezig \u2013 grondwater < 1 m of onbekend  \u2192  f_gw = 1,15', 1.15),
    ],
    description='Grondwaterstand:', layout=_W_WIDE, style=_LBL)
gr_rc = widgets.BoundedFloatText(value=3.5, min=0.01, max=20, step=0.1,
    description='R_c [m\u00b2\u00b7K/W]:', layout=_W_MED, style=_LBL)
gr_area = widgets.BoundedFloatText(value=10, min=0, max=10000, step=0.5,
    description='Oppervlak A [m\u00b2]:', layout=_W_MED, style=_LBL)

# ── Dynamic panel & output ───────────────────────────────────────────────────
fields_box = widgets.VBox([])
fk_out = widgets.Output()


def _build_fields(scenario):
    """Return the ordered list of widgets for the active scenario."""
    common = [widgets.HBox([theta_i_input, theta_e_input])]

    if scenario == 'Buitenlucht':
        return common + [
            widgets.HBox([buitenlucht_bouwdeel, buitenlucht_heated]),
            buitenlucht_hs,
        ]

    if scenario == 'Aangrenzend gebouw':
        return common + [
            widgets.HBox([ag_bouwdeel, ag_heated]),
            ag_theta_b,
            ag_hs,
        ]

    if scenario == 'Verwarmde ruimte (zelfde woning)':
        return common + [
            widgets.HBox([vr_bouwdeel, vr_heated]),
            vr_theta_a,
            widgets.HBox([vr_override, vr_theta_manual]),
            vr_hs_own,
            vr_hs_adj,
        ]

    if scenario == 'Onverwarmde ruimte \u2013 bekende temperatuur':
        return common + [
            widgets.HBox([ob_bouwdeel, ob_heated]),
            ob_theta_a,
            ob_hs,
        ]

    if scenario == 'Onverwarmde ruimte \u2013 onbekende temperatuur':
        return [
            oo_doel,
            oo_ruimte,
            oo_gevels,
            oo_daktype,
            oo_buitenwanden,
            oo_ventilatievoud,
            oo_a_opening,
            oo_opening_mm2,
            oo_tijdconst_ruimte,
        ]

    if scenario == 'Grond':
        return common + [
            widgets.HBox([gr_bouwdeel, gr_heated]),
            gr_theta_me,
            gr_hs,
            gr_gwstand,
            gr_rc,
            gr_area,
        ]

    return []


def _on_scenario_change(_=None):
    fields_box.children = _build_fields(scenario_dd.value)
    _compute()


def _compute(_=None):
    """Run the appropriate fk_calc function and render results."""
    with fk_out:
        fk_out.clear_output(wait=True)
        s = scenario_dd.value
        try:
            rows = []

            if s == 'Buitenlucht':
                bd_map = {
                    'Buitenwand':              'buitenwand',
                    'Schuin dak':              'schuin_dak',
                    'Vloer boven buitenlucht': 'vloer_boven_buitenlucht',
                    'Plat dak':                'plat_dak',
                }
                bd = bd_map[buitenlucht_bouwdeel.value]
                hs_id = hs_options[buitenlucht_hs.value] if bd in (
                    'vloer_boven_buitenlucht', 'plat_dak') else None
                f = fk_calc.calc_f_k_buitenlucht(
                    bd, theta_i_input.value, theta_e_input.value,
                    hs_id, buitenlucht_heated.value)
                rows.append(('f_k', f'{f:.4f}'))

            elif s == 'Aangrenzend gebouw':
                bd = ag_bouwdeel.value.lower()
                hs_id = hs_options[ag_hs.value] if bd in ('vloer', 'plafond') else None
                f = fk_calc.calc_f_ia_k_aangrenzend_gebouw(
                    bd, theta_i_input.value, theta_e_input.value,
                    ag_theta_b.value, hs_id, ag_heated.value)
                rows.append(('f_ia,k', f'{f:.4f}'))

            elif s == 'Verwarmde ruimte (zelfde woning)':
                bd = vr_bouwdeel.value.lower()
                # Use manual override if checked, otherwise use dropdown value
                theta_a = vr_theta_manual.value if vr_override.value else vr_theta_a.value
                hs_own = hs_options[vr_hs_own.value] if bd != 'wand' else None
                hs_adj = hs_options[vr_hs_adj.value] if bd != 'wand' else None
                f = fk_calc.calc_f_ia_k_verwarmde_ruimte(
                    bd, theta_i_input.value, theta_e_input.value,
                    theta_a, hs_own, hs_adj, vr_heated.value)
                rows.append(('f_ia,k', f'{f:.4f}'))

            elif s == 'Onverwarmde ruimte \u2013 bekende temperatuur':
                bd = ob_bouwdeel.value.lower()
                hs_id = hs_options[ob_hs.value] if bd in ('vloer', 'plafond') else None
                f = fk_calc.calc_f_k_onverwarmd_bekend(
                    bd, theta_i_input.value, theta_e_input.value,
                    ob_theta_a.value, hs_id, ob_heated.value)
                rows.append(('f_k', f'{f:.4f}'))

            elif s == 'Onverwarmde ruimte \u2013 onbekende temperatuur':
                if oo_doel.value == 'Warmteverlies':
                    rt_map = {
                        'Vertrek':          'vertrek',
                        'Ruimte onder dak': 'dak',
                        'Verkeersruimte':   'verkeersruimte',
                        'Kruipruimte':      'kruipruimte',
                    }
                    rt = rt_map[oo_ruimte.value]
                    kwargs = {}
                    if rt == 'vertrek':
                        n_gevels, buitendeur = oo_gevels.value
                        kwargs['aantal_externe_gevels'] = n_gevels
                        if buitendeur is not None:
                            kwargs['buitendeur_aanwezig'] = buitendeur
                    elif rt == 'dak':
                        kwargs['daktype'] = oo_daktype.value
                    elif rt == 'verkeersruimte':
                        kwargs['heeft_buitenwanden'] = oo_buitenwanden.value
                        kwargs['ventilatievoud']      = oo_ventilatievoud.value
                        kwargs['a_opening_per_v']     = oo_a_opening.value
                    elif rt == 'kruipruimte':
                        kwargs['openingsgrootte_mm2_per_m2'] = oo_opening_mm2.value
                    f = fk_calc.calc_f_k_onverwarmd_onbekend_warmteverlies(rt, **kwargs)
                    rows.append(('f_k (Tabel 2.3)', f'{f:.2f}'))
                else:
                    f = fk_calc.calc_f_k_onverwarmd_onbekend_tijdconstante(
                        oo_tijdconst_ruimte.value)
                    rows.append(('f_k (Tabel 2.13)', f'{f:.2f}'))

            elif s == 'Grond':
                bd    = gr_bouwdeel.value.lower()
                hs_id = hs_options[gr_hs.value] if bd == 'vloer' else None
                f_ig  = fk_calc.calc_f_ig_k(
                    bd, theta_i_input.value, theta_e_input.value,
                    gr_theta_me.value, hs_id, gr_heated.value)
                # f_gw is stored directly as 1.00 or 1.15 in the dropdown value
                f_gw  = gr_gwstand.value
                u_eq  = fk_calc.calc_u_equiv_k(gr_rc.value)
                h_t   = gr_area.value * u_eq * f_ig * f_gw
                rows.append(('f_ig,k (formule)',               f'{f_ig:.4f}'))
                rows.append(('f_gw',                           f'{f_gw:.2f}'))
                rows.append(('U_equiv,k [W/(m\u00b2\u00b7K)]', f'{u_eq:.2f}'))
                rows.append(('H_T,ig [W/K]',                   f'{h_t:.3f}'))

            trs = ''.join(
                f'<tr><td style="padding:4px 12px;font-weight:bold">{lbl}</td>'
                f'<td style="padding:4px 12px">{val}</td></tr>'
                for lbl, val in rows
            )
            display(HTML(
                '<table style="border-collapse:collapse;font-family:sans-serif;'
                'font-size:14px;margin-top:8px">'
                '<tr>'
                '<th style="background:#2c5f8a;color:white;padding:6px 12px;text-align:left">Factor</th>'
                '<th style="background:#2c5f8a;color:white;padding:6px 12px;text-align:left">Waarde</th>'
                '</tr>'
                f'{trs}</table>'
            ))

        except Exception as exc:
            display(HTML(
                f'<p style="color:#c00;font-weight:bold">\u26a0 {exc}</p>'))


# ── Wire all widget events ────────────────────────────────────────────────────
scenario_dd.observe(_on_scenario_change, names='value')
for w in [
    theta_i_input, theta_e_input,
    buitenlucht_bouwdeel, buitenlucht_hs, buitenlucht_heated,
    ag_bouwdeel, ag_theta_b, ag_hs, ag_heated,
    vr_bouwdeel, vr_theta_a, vr_override, vr_theta_manual,
    vr_hs_own, vr_hs_adj, vr_heated,
    ob_bouwdeel, ob_theta_a, ob_hs, ob_heated,
    oo_doel, oo_ruimte, oo_gevels, oo_daktype,
    oo_buitenwanden, oo_ventilatievoud, oo_a_opening,
    oo_opening_mm2, oo_tijdconst_ruimte,
    gr_bouwdeel, gr_theta_me, gr_hs, gr_heated,
    gr_gwstand, gr_rc, gr_area,
]:
    w.observe(_compute, names='value')

# ── Display ───────────────────────────────────────────────────────────────────
display(widgets.VBox([
    widgets.HTML('<b>Kies aangrenzende situatie</b>'),
    scenario_dd,
    widgets.HTML('<b style="margin-top:8px;display:block">Invoervelden</b>'),
    fields_box,
    widgets.HTML('<b style="margin-top:8px;display:block">Resultaat</b>'),
    fk_out,
]))

_on_scenario_change()  # initialize
